In [1]:
import json
import sys
from collections import Counter
from dataclasses import dataclass
from typing import List, Tuple

import cv2
import networkx as nx
import numpy as np
import scipy.ndimage.measurements
import shapely.geometry
from PIL import Image
from skimage import morphology, segmentation

from extract_network import find_color, extract_network, render_skeleton, render_network

In [2]:
rgb = (255, 255, 255)
png_file = 'samples/gt/0_gray.png'
im = Image.open(png_file)

In [3]:
px = find_color(im, rgb).T
g = extract_network(px, 16)
print(f'Extracted street network:')
print(f'  - {len(g.nodes())} nodes')
print(f'  - {len(g.edges())} edges')

skel = morphology.skeletonize(px)
skel_path = png_file.replace('.png', '.skel.png')
img_skeleton = render_skeleton(im, skel, (255, 0, 0))
img_skeleton.show()
# print(f'Wrote {skel_path}')

Skeleton px=1718
Adding 0/0 dense nodes to existing 22 nodes.
Merged (246, 246) and (246, 248), d=2
Merged (256, 239) and (256, 241), d=2
Merged (256, 254) and (256, 256), d=2
Merged (246, 247) and (256, 240), d=10
Merged (251, 243) and (256, 255), d=15
Extracted street network:
  - 17 nodes
  - 18 edges
(512, 512)


In [4]:
out_path = png_file.replace('.png', '.grid.png')
img_network = render_network(im, g, (0, 255, 0))
img_network.show()
# print(f'Wrote {out_path}')

graph nodes: 17
graph edges: 18


In [5]:
# fc = network_to_geojson(g)
# geojson_path = png_file.replace('.png', '.grid.geojson')
# with open(geojson_path, 'w') as out:
#     json.dump(fc, out)
# print(f'Wrote {geojson_path}')